# Data Reading

In [1]:
import pandas as pd

In [2]:
def load_data(prefix):
    def fix_path(src):
        return prefix + str('/IMG/') +src.split('/')[-1]    
    
    data = pd.read_csv( prefix + str('/driving_log.csv'), header=None, skiprows=1, names=[
        'center_img_path', 
        'left_img_path',
        'right_img_path',
        'steering_angle',
        'throttle',
        'break',
        'speed'
    ])
    
    data['center_img_path'] =  data['center_img_path'].apply(fix_path)
    data['left_img_path'] =  data['left_img_path'].apply(fix_path)
    data['right_img_path'] =  data['right_img_path'].apply(fix_path)

    return data

In [3]:
datasets = [load_data(prefix) for prefix in ['./train', './train_2', './data'] ]

In [4]:
data = pd.concat( datasets )
data.head(3)

,center_img_path,left_img_path,right_img_path,steering_angle,throttle,break,speed
0,./train/IMG/center_2017_07_26_18_08_08_545.jpg,./train/IMG/left_2017_07_26_18_08_08_545.jpg,./train/IMG/right_2017_07_26_18_08_08_545.jpg,0.0,0.0,0.0,8.660496e-07
1,./train/IMG/center_2017_07_26_18_08_08_623.jpg,./train/IMG/left_2017_07_26_18_08_08_623.jpg,./train/IMG/right_2017_07_26_18_08_08_623.jpg,0.0,0.0,0.0,1.452814e-06
2,./train/IMG/center_2017_07_26_18_08_08_696.jpg,./train/IMG/left_2017_07_26_18_08_08_696.jpg,./train/IMG/right_2017_07_26_18_08_08_696.jpg,0.0,0.0,0.0,1.847686e-06


In [5]:
data.to_pickle( 'loaded_data.pkl' )

# Data Generation

In [6]:
import pandas as pd
import numpy as np
import cv2


def generator(batch_size, data):
    def load_img(src):
        return cv2.imread(src)

    correction = 0.2
    while True:
        subsample = data.sample(batch_size)
        
        subsample['center_img'] = subsample.center_img_path.apply(load_img)
        subsample['left_img'] = subsample.left_img_path.apply(load_img)
        subsample['right_img'] = subsample.right_img_path.apply(load_img)
    
        
        X_center = np.asarray( subsample.center_img.values.tolist() )
        y_center = subsample[['steering_angle']].values

        X_left = np.asarray( subsample.left_img.values.tolist() )
        y_left = subsample[['steering_angle']].values + correction

        X_right = np.asarray( subsample.right_img.values.tolist() )
        y_right = subsample[['steering_angle']].values - correction

        X = np.concatenate( [X_center, X_left, X_right], axis=0)
        y = np.concatenate( [y_center, y_left, y_right], axis=0)
        
        yield X, y

In [7]:
data = pd.read_pickle( 'loaded_data.pkl' )

validation_data = data.sample( int(data.shape[0] * 0.1) )
train_data = data.drop(validation_data.index)

#X_valid, y_valid = next(generator(validation_data.shape[0], validation_data))


# Neural Net

In [8]:
import os

In [9]:
def preprocess_image(img):
    from keras.backend import tf as ktf
    import keras.backend as K
    
    ktf.image.resize_images(img, (80, 160))
    return (K.identity(img) / 255.0) - 0.5


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.applications import VGG16
from keras.regularizers import l2

tb = keras.callbacks.TensorBoard(log_dir='./logs')

vgg = VGG16( include_top=False, weights='imagenet' )
vgg.trainable = False

model = Sequential()
model.add( Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)) )
model.add( Lambda(preprocess_image) )
model.add( vgg )
model.add( Flatten() )
model.add( Dense(1024, activation='relu', kernel_regularizer=l2(0.0001)  ))
model.add( Dense(512, activation='relu', kernel_regularizer=l2(0.0001) ))
model.add( Dropout(0.5))
model.add( Dense(1) )

model.compile( optimizer='adam', loss='mse' )
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 10240)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              10486784  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________

In [ ]:
batch_size=64
model.fit_generator( generator=generator(batch_size, train_data), 
                    steps_per_epoch=200, 
                    epochs=100, 
                    callbacks=[tb],
                    validation_data=generator(batch_size, validation_data),
                    validation_steps=20
                   )


Epoch 1/100
200/200 [==============================] - 166s - loss: 1.8040 - val_loss: 0.1438
Epoch 2/100
200/200 [==============================] - 157s - loss: 0.1295 - val_loss: 0.1184
Epoch 3/100
200/200 [==============================] - 157s - loss: 0.1122 - val_loss: 0.1057
Epoch 4/100
200/200 [==============================] - 165s - loss: 0.1007 - val_loss: 0.0939
Epoch 5/100
200/200 [==============================] - 168s - loss: 0.0924 - val_loss: 0.0878
Epoch 6/100
200/200 [==============================] - 164s - loss: 0.0862 - val_loss: 0.0839
Epoch 7/100
200/200 [==============================] - 158s - loss: 0.0801 - val_loss: 0.0791
Epoch 8/100
200/200 [==============================] - 156s - loss: 0.0756 - val_loss: 0.0739
Epoch 9/100
199/200 [============================>.] - ETA: 0s - loss: 0.0719

In [ ]:
model.save( '2017-07-26.h5' )